## Norms

In [1]:
def integralNorm(function,axis):
    return np.sqrt(scipy.integrate.simps(function**2,axis))

def l2norm(values):
    sum_sq = sum(values**2)
    return np.sqrt( sum_sq / len(values))

## Error and convergence

In [2]:
def errorPlot(method, function, solution, alpha=None, sigma=None, startindex=1, endindex=4, returnvalue=False, contNorm = True, expected_order=2):
        '''
        Plot a loglog convergence plot,
        '''
        ms = np.logspace(startindex, endindex, 10, dtype=np.intc)
        n = 2*max(ms)
        
        if(contNorm):
            errorsdisc = np.zeros(len(ms)) #Stores the error for the l_2 norm,
            errorscont = np.zeros(len(ms)) #Strores the error for the L_2 norm,
            i=0
            for m in ms:
                if alpha==None or sigma==None:
                    axis, sol = method(m,function, n = n) #Obtain numeric solution,
                else:
                    axis, sol = method(m,alpha,sigma,function) #Obtain numeric solution,
                errorsdisc[i] = np.linalg.norm(sol-solution(axis))/np.linalg.norm(solution(axis)) #Calculate error,
                errorscont[i] = integralNorm(sol-solution(axis),axis)/integralNorm(solution(axis),axis)
                i+=1
            
            #Plotting,
            fig, ax = plt.subplots()
            
            ax.loglog(ms,errorsdisc,label ="$\ell_2$-norm", marker = ".")
            ax.loglog(ms,errorscont, linestyle=(0,(5,10)),label="$L_2$-norm",color="red", marker = ".")
            ax.plot(np.logspace(np.log10(ms[0]), np.log10(ms[-1]), 5), 
                    np.logspace(np.log10(errorsdisc[0]), np.log10(errorsdisc[0])-
                                expected_order*(np.log10(ms[-1])-np.log10(ms[0])), 5), "s", label="Expected order")
            
            ax.legend()
            ax.set_xlabel("Number of points along x-axis: $M$")
            ax.set_ylabel("Relative error $e^r_{(\cdot)}$")
            ax.grid()
            plt.show()    
            if returnvalue:
                return ms, errorsdisc, errorscont
        else: #If we dont want to check the continous L2 norm,
            errorsdisc = np.zeros(len(ms)) #Stores the error for the l_2 norm,
            i=0
            for m in ms:
                if alpha==None or sigma==None:
                    axis, sol = method(m,function) #Obtain numeric solution,
                else:
                    axis, sol = method(m,alpha,sigma,function) #Obtain numeric solution,
                errorsdisc[i] = np.linalg.norm(sol-solution(axis))/np.linalg.norm(solution(axis)) #Calculate error,
                i+=1
    
            #Plotting,
            plt.loglog(ms,errorsdisc,label ="disc")
            plt.grid()
            plt.legend()
            plt.xlabel("number of points M")
            plt.ylabel("Errors $e^r_{(\cdot)}$")
            plt.title("Loglog plot the error")
            
            plt.show()    
            if returnvalue:
                return ms, errorsdisc

In [3]:
def estimateOrder(xvec, yvec, sigfigs = 3):
        '''
        Estimate order of a convergence plot
        '''
        
        if(len(xvec)!=len(yvec)):
            raise Exception("the inputs do not have the same dimension")
        ylog = np.log(yvec)
        xlog = np.log(xvec)
        return np.abs( (ylog[-1]-ylog[0]) / (xlog[-1]-xlog[0]) )
    
def expectedOrder(startx, endx, starty, order, resolution=5):
    '''
    Make a set of x-values and y-values to use for plotting expected order of convergence.
    Use like this: plt.loglog(*expectedOrder(startx endx starty order))

    startx     : (float) x-value you want the line to start at
    endx       : (float) x-value you want the line to end at
    starty     : (float) y-value you want the line to start at
    order      : (float) expected order
    resolution : (float) number of points in the line

    returns    : (logspace)(logspace)
    '''
    return np.logspace(np.log10(startx), np.log10(endx), resolution, base=10), np.logspace(np.log10(starty), np.log10(starty)-order*(np.log10(endx)-np.log10(startx)), resolution, base=10)

In [4]:
#A plotting function that takes in a matrix or function along with axies for it to be plotted with
def plot3D(solution, xaxis, yaxis, title ="", xlabel="x", ylabel="y", zlabel="Output", zlim = None, xlim = None, ylim = None, vmin = None, vmax = None):
    
        #Generate mesh-axes
        x = np.outer(xaxis, np.ones(len(yaxis)))
        t = np.outer(yaxis, np.ones_like(xaxis)).T
    
        #Generate figure objects
        fig = plt.figure()
        ax = plt.axes(projection='3d')
    
        #If the solution is a callable func, we want to call it with our mesh axies. If not, plot the matrix.
        if callable(solution):
            if(vmin != None and vmax != None):
                ax.plot_surface(x, t, solution(x,t), cmap='viridis', vmin = vmin, vmax = vmax,edgecolor='none', zorder=1)
            else:
                ax.plot_surface(x, t, solution(x,t), cmap='viridis', edgecolor='none', zorder=1)
        else:
            if x.shape!=t.shape or x.shape!=solution.shape:
                print("Shapes are not correct! x:", x.shape, "y:", t.shape, "Solution", solution.shape,"\nCannot plot.")
            if(vmin != None and vmax != None):
                ax.plot_surface(x, t, solution, cmap='viridis', vmin = vmin, vmax = vmax,edgecolor='none', zorder=1)
            else:
                ax.plot_surface(x, t, solution,cmap='viridis', edgecolor='none', zorder=1)
        
        #Add some limits to the axies if specified
        if zlim!=None:
            ax.set_zlim(zlim)
        if xlim!=None:
            ax.set_xlim(xlim)
        if ylim!=None:
            ax.set_ylim(ylim)
        
        #Add titles and lables
        ax.set_title(title)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_zlabel(zlabel)
        
        #Dispaly the plot
        plt.show()